In [ ]:
## Load the model

In [ ]:
import sys
import ast
sys.path.append("..")
sys.path.append("../src")

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import segmentation_models as sm
sm.set_framework('keras')
import ad_seg_utils as seg_utils
import bb_eval_utils as eval_utils
import csv
import math
import albumentations as A

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [ ]:
SEGMENTATION = "AD"
PERTURBATION = "base"
print("Program initiating... type of segmentation: " + SEGMENTATION + ", type of perturbation: " + PERTURBATION)

BIN_SEG = False
CLASSES = ['background', 'skin']
WEIGHTS = np.array([1, 1])
target_idx = 1
MODEL_NAME = '/skin_base_ce_bestOnly/base_skin_ce.h5'
BACKBONE = 'efficientnetb3'
preprocess_input = sm.get_preprocessing(BACKBONE)

# set parameters based on the type of segmentation
if SEGMENTATION == 'SKIN' or SEGMENTATION == 'skin':
    pass
elif SEGMENTATION == 'AD' or SEGMENTATION == 'ad':
    BIN_SEG = False
    target_idx = 2
    CLASSES = ['background', 'skin', 'eczema']
    WEIGHTS = np.array([1, 1, 1])
    MODEL_NAME = '/ad_base_ce_bestOnly/base_ad_ce.h5'
else:
    print('Unexpected type of segmentation, should be either skin or ad\n program terminated')



"""# Model Evaluation"""
# config PROJ_DIR according to your environment
PROJ_DIR = "/path_to_project_dir"
PRED_DIR = os.path.join(PROJ_DIR, 'output/predictions/' + "test")
BB_DIR = os.path.join(PROJ_DIR, 'output/bounding_boxes')
EVAL_DIR = os.path.join(PROJ_DIR, 'output/evaluations')
MODEL_DIR = os.path.join(PROJ_DIR, 'output/models')
DATA_DIR = os.path.join(PROJ_DIR, 'data')

# new dataset
x_test_dir = os.path.join(DATA_DIR, 'test_set/reals')
y_test_dir = os.path.join(DATA_DIR, 'test_set/labels')

print('reading test images from: ' + str(x_test_dir))
print('reading test masks from: ' + str(y_test_dir))

test_dataset = seg_utils.Dataset(
    x_test_dir, 
    y_test_dir, 
    classes=CLASSES, 
    augmentation=None,
    preprocessing=seg_utils.get_preprocessing(preprocess_input),
    is_train=False,
    use_full_resolution=False,
    binary_seg=BIN_SEG,
)
eczema_dataset = seg_utils.Dataset(
    x_test_dir, 
    y_test_dir, 
    classes=['background', 'skin', 'eczema'],
    augmentation=None,
    preprocessing=seg_utils.get_preprocessing(preprocess_input),
    is_train=False,
    use_full_resolution=False,
    binary_seg=0,
)


model = seg_utils.load_model(dir=MODEL_DIR+MODEL_NAME, classes=CLASSES, weights=WEIGHTS)
print('Trained model loaded!')

## Example to predict mask, generate box and produce square crops

In [ ]:
# %%time
folder_dir = '/path_to_proj_dir/data/test_set/reals'
fns = os.listdir(folder_dir)

for file in fns[1:10]:
    print(os.path.join(folder_dir, file))
    seg_utils.run_sigle_pred(model, os.path.join(folder_dir, file), "/path_to_proj_dir/output/predictions/test", file, target_idx=target_idx, resize_ratio=0.4, preprocessing=seg_utils.get_preprocessing(preprocess_input), refno="0000",visno="0")
